Rozwiązywanie układów równań liniowych metodą iteracyjną Jacobiego

Wykorzystane moduły

In [2]:
import numpy as np
from enum import Enum
from functools import wraps
import time

![title](img_vsc/img01.png)

In [3]:
# Kryteria przyrostowe
class Condition(Enum):
    
    FIRST = lambda X_approx,  X, eps: np.linalg.norm(X_approx - X) < eps 
    SECOND = lambda A, X, b, eps: np.linalg.norm(A @ X - b) < eps

In [4]:
def measure_jacobi_time(func):
    
    @wraps(func)
    def wrapper(*args):
        
        start = time.perf_counter()
        
        X, it = func(*args)
        
        end = time.perf_counter()

        return X, it, end - start
    return wrapper

In [5]:
class JacobiTest():
    
    k = 8
    m = 2
    
    @staticmethod
    def get_start_vector_1(n):
        return np.zeros(n).astype(np.float64)
    
    @staticmethod
    def get_start_vector_2(n):
        return np.array([-500 if i % 2 == 0 else 500 for i in range(n)], dtype = np.float64)
    
    @measure_jacobi_time
    @staticmethod
    def jacobi(A, b, condition, eps, start_vector, max_iterations):
        
        D = np.diag(A).astype(np.float64) # Przekątna macierzy A
        C = A - np.diagflat(D).astype(np.float64) # A bez przekątnej
        X = start_vector(b.shape[0])
        
        for it in range(max_iterations):
            X_approx = (b - (C @ X)) / D
            
            if condition == Condition.FIRST and Condition.FIRST(X_approx, X, eps):
                break
            elif condition == Condition.SECOND and Condition.SECOND(A, X, b, eps):
                break
            X = X_approx
            
        return X, it
    
    @classmethod
    def __get_matrix(cls, n):
        return np.array([[cls.k if i == j else 1 / (abs(i - j) + cls.m) for j in range(n)] for i in range(n)], dtype=np.float64)
    
    @staticmethod
    def __get_vector(n):
        return np.array([1 if i % 2 == 0 else -1 for i in range(n)], dtype=np.float64)
        
    @classmethod
    def get_system(cls, n):
        
        A = cls.__get_matrix(n)
        X = cls.__get_vector(n)
        
        return A, A @ X, X
    
    @classmethod
    def jacobi_n(cls, systems, eps, condition, start_vector, max_iterations = 1000):
        
        result = []
        
        for A, b, X in systems:
                        
            X_approx, iterations, duration = cls.jacobi(A, b, condition, eps, start_vector, max_iterations)
                    
            result.append((np.linalg.norm(X - X_approx), iterations, duration))
            
        return result      

In [6]:
n = [3, 5, 7, 10, 15, 20, 35, 50, 75, 100, 125, 150, 200, 250, 300, 350]
eps = [1e-3, 1e-7, 1e-15, 1e-20]
systems = list(map(lambda x : JacobiTest().get_system(x), n))

Testy dla wektora początkowego złożonego z samych zer

Precyzja 1e-3

In [10]:
results_first_1e_3 = JacobiTest.jacobi_n(systems, 1e-3, Condition.FIRST, JacobiTest.get_start_vector_1, 1000)
results_second_1e_3 = JacobiTest.jacobi_n(systems, 1e-3, Condition.SECOND, JacobiTest.get_start_vector_1, 1000)

In [11]:
for i, (error, iterations, duration) in enumerate(results_first_1e_3):
    print(n[i], error, iterations, duration)

3 0.00027657023415481746 3 0.00015959999996084662
5 0.00011797844999045876 4 0.00010220000001481822
7 0.00030706141805554826 4 9.480000005623879e-05
10 0.0003370163472400488 3 7.900000002791785e-05
15 0.00047669834508336456 5 0.00012040000001434237
20 0.000651468788308314 3 9.699999998247222e-05
35 0.0005579595453482399 7 0.00014620000001741573
50 0.0006949789352982175 4 0.0001504000000522865
75 0.000493832536680288 11 0.00025670000002264715
100 0.00039873796638740367 6 0.0019119999999475112
125 0.0004500863757870157 17 0.00376240000002781
150 0.000494924079886817 7 0.0019761999999445834
200 0.0005521724962843907 8 0.0016976999999087639
250 0.0005999514339342845 9 0.0017288000000235115
300 0.000439484401750828 11 0.0034300999999459236
350 0.0005070354198577723 12 0.004392000000052576


In [12]:
for i, (error, iterations, duration) in enumerate(results_second_1e_3):
    print(n[i], error, iterations, duration)

3 1.892207222489987e-05 4 9.179999995012622e-05
5 1.5671221205628294e-05 5 7.870000001730659e-05
7 5.4797787725071414e-05 5 0.00011279999989710632
10 1.6083207913400562e-05 4 6.08000000283937e-05
15 4.3848971999910014e-05 7 0.0001312000000552871
20 6.0163907038124154e-05 4 6.229999996776314e-05
35 5.8395157059540475e-05 10 0.0001081000000340282
50 5.561585132904874e-05 6 0.0001002999999855092
75 5.3117864274722075e-05 16 0.0004327999999986787
100 7.196233810393804e-05 8 0.0024402000000236512
125 6.556866221638224e-05 24 0.008125500000005559
150 6.737289647735507e-05 10 0.0030751000000464046
200 6.256286635347576e-05 12 0.002668500000027052
250 6.056900370161495e-05 14 0.00393860000008317
300 6.166575360887368e-05 16 0.0053371000000197455
350 6.595897929457682e-05 18 0.005415500000026441


Precyzja 1e-7

In [36]:
results_first_1e_7 = JacobiTest.jacobi_n(systems, 1e-7, Condition.FIRST, JacobiTest.get_start_vector_1, 1000)
results_second_1e_7 = JacobiTest.jacobi_n(systems, 1e-7, Condition.SECOND, JacobiTest.get_start_vector_1, 1000)

In [37]:
for i, (error, iterations, duration) in enumerate(results_first_1e_7):
    print(n[i], error, iterations, duration)

3 7.925528275653803e-09 7 0.00018199999999524152
5 3.708892142531895e-08 8 0.00014119999991635268
7 5.576016066291517e-08 9 0.00014930000008916977
10 3.673605066208856e-08 6 8.120000006783812e-05
15 3.4127886906484605e-08 13 0.00018520000003263704
20 1.5229267214950545e-08 8 8.309999998346029e-05
35 6.694208212390076e-08 19 0.00016470000002755114
50 2.8773489493222252e-08 12 0.00013759999990270444
75 4.232095926329985e-08 32 0.000512599999979102
100 3.2589341528906207e-08 17 0.002553999999918233
125 5.120667635624557e-08 50 0.009849700000017947
150 4.518212555072474e-08 21 0.004055399999970177
200 5.2989810336152925e-08 25 0.004163899999980458
250 3.951210815641748e-08 30 0.006124099999965438
300 5.24963999093285e-08 34 0.0068128999999999
350 5.2419108850599144e-08 39 0.0067210999999360865


In [38]:
for i, (error, iterations, duration) in enumerate(results_second_1e_7):
    print(n[i], error, iterations, duration)

3 7.925528275653803e-09 7 0.00011100000006081245
5 4.943431302229421e-09 9 9.569999997438572e-05
7 9.959123360247537e-09 10 0.00010229999998045969
10 1.7574842295615031e-09 7 7.909999999355932e-05
15 3.139251112367329e-09 15 0.00014269999996940896
20 1.9610433428683717e-09 9 9.810000005927577e-05
35 7.006051855814652e-09 22 0.00021200000003318564
50 8.152635404040681e-09 13 0.00015340000004471221
75 7.110239489259351e-09 36 0.0003968999999415246
100 5.888064149850948e-09 19 0.005084000000010747
125 5.665168605223369e-09 58 0.014679000000000997
150 6.158472188026357e-09 24 0.006545300000084353
200 6.0107683411578005e-09 29 0.006906599999979335
250 6.314700378720415e-09 34 0.007710999999972046
300 7.368462017964859e-09 39 0.009004799999956958
350 6.820836237137304e-09 45 0.017991800000004332


Precyzja 1e-15

In [39]:
results_first_1e_15 = JacobiTest.jacobi_n(systems, 1e-15, Condition.FIRST, JacobiTest.get_start_vector_1, 1000)
results_second_1e_15 = JacobiTest.jacobi_n(systems, 1e-15, Condition.SECOND, JacobiTest.get_start_vector_1, 1000)

In [40]:
for i, (error, iterations, duration) in enumerate(results_first_1e_15):
    print(n[i], error, iterations, duration)

3 2.220446049250313e-16 14 0.00037229999998089625
5 5.551115123125783e-16 17 0.0001672999999300373
7 4.1540741810552243e-16 20 0.00022769999998217827
10 6.080941944488118e-16 12 0.00011789999996381084
15 7.771561172376096e-16 28 0.00021590000005744514
20 5.978733960281817e-16 17 0.00019550000001800072
35 1.3642648929939613e-15 44 0.0006975999999667692
50 1.7728839976828627e-15 27 0.0004663999999365842
75 1.9860273225978185e-15 72 0.0006336000000146669
100 1.877557501171774e-15 38 0.00596090000010463
125 2.3733857464271978e-15 113 0.017305399999941073
150 3.7105374255538066e-15 50 0.006133299999987685
200 5.074338674593921e-15 62 0.007236000000034437
250 6.654858695507619e-15 80 0.010913200000004508
300 8.015933253251316e-15 94 0.014244299999973009
350 9.222205069512406e-15 999 0.12481609999997545


In [41]:
for i, (error, iterations, duration) in enumerate(results_second_1e_15):
    print(n[i], error, iterations, duration)

3 0.0 15 0.00017600000001039007
5 1.5700924586837752e-16 18 0.00016749999997500709
7 1.1102230246251565e-16 22 0.00020609999990028882
10 2.7194799110210365e-16 13 0.00014999999996234692
15 5.324442579404919e-16 999 0.007287300000029973
20 6.568167990716596e-16 999 0.007321799999999712
35 1.2412670766236366e-15 999 0.0078038999999989755
50 1.7199501139797033e-15 999 0.009071300000073279
75 1.870981097424087e-15 999 0.009794999999940046
100 1.7832822334594722e-15 999 0.19904270000006363
125 2.336749147496729e-15 999 0.21004049999999097
150 3.66709875019605e-15 999 0.21679519999997865
200 5.056087786996822e-15 999 0.20574729999998453
250 6.650226650194185e-15 999 0.21128900000007889
300 8.015933253251316e-15 999 0.2098747000000003
350 9.222205069512406e-15 999 0.21632429999999658


Wektor początkowy złożony z 500 i -500 na przemian

Precyzja 1e-3

In [43]:
results_first_1e_3 = JacobiTest.jacobi_n(systems, 1e-3, Condition.FIRST, JacobiTest.get_start_vector_2, 1000)
results_second_1e_3 = JacobiTest.jacobi_n(systems, 1e-3, Condition.SECOND, JacobiTest.get_start_vector_2, 1000)

In [44]:
for i, (error, iterations, duration) in enumerate(results_first_1e_3):
    print(n[i], error, iterations, duration)

3 0.0006930975717296203 5 0.0001594999998815183
5 0.00013941201150584524 7 7.74000000092201e-05
7 0.00015641004915180996 8 0.00015690000009271898
10 0.0003849445574737385 5 0.00014609999993808742
15 0.0006128753194155822 10 0.00012169999990874203
20 0.00046091193947575895 6 8.029999980863067e-05
35 0.0003204518728001519 16 0.0001526999999441614
50 0.0006337445083183972 9 0.00010300000008101051
75 0.0004809145989144053 25 0.000230699999974604
100 0.0005001724466855992 13 0.0024693000000297616
125 0.0005294448646467454 39 0.007577800000035495
150 0.0006270382660380517 16 0.0031853000000410248
200 0.0004032813191235235 20 0.0032489000000168744
250 0.0004900365717379591 23 0.004257300000062969
300 0.00041098028576843845 27 0.0062347000000499975
350 0.00055950678440798 30 0.005010599999877741


In [45]:
for i, (error, iterations, duration) in enumerate(results_second_1e_3):
    print(n[i], error, iterations, duration)

3 5.216923946389555e-05 6 9.789999990061915e-05
5 1.8581549631552e-05 8 9.469999986322364e-05
7 2.7935840495660232e-05 9 9.959999988495838e-05
10 1.8404761317416775e-05 6 7.589999995616381e-05
15 5.6375290570531125e-05 12 0.00013650000005327456
20 5.926344612600765e-05 7 9.020000015880214e-05
35 7.116703949669136e-05 18 0.00019249999991188815
50 5.087738529040477e-05 11 0.00014620000001741573
75 5.172837868799123e-05 30 0.00036160000013296667
100 3.841181698424909e-05 16 0.003872100000080536
125 5.857428225575343e-05 47 0.009760599999935948
150 4.398476526162751e-05 20 0.004400100000111706
200 7.882451079433622e-05 23 0.005152299999963361
250 4.951726591292597e-05 28 0.006810699999959979
300 5.768571996922662e-05 32 0.008090899999842804
350 7.280368454189154e-05 36 0.008879799999931492


Precyzja 1e-7

In [46]:
results_first_1e_7 = JacobiTest.jacobi_n(systems, 1e-7, Condition.FIRST, JacobiTest.get_start_vector_2, 1000)
results_second_1e_7 = JacobiTest.jacobi_n(systems, 1e-7, Condition.SECOND, JacobiTest.get_start_vector_2, 1000)

In [47]:
for i, (error, iterations, duration) in enumerate(results_first_1e_7):
    print(n[i], error, iterations, duration)

3 2.3221199190271556e-08 9 0.00035530000013750396
5 4.39982610801569e-08 11 0.0002993999999034713
7 2.842825076659887e-08 13 0.00037950000000819273
10 4.214451718954487e-08 8 0.00028029999998580024
15 4.3877186973263134e-08 18 0.0007147000001168635
20 1.6291637649923376e-08 11 0.0005034999999224965
35 3.844672296925235e-08 28 0.00044870000010632793
50 2.6324353162588323e-08 17 0.0003696000001127686
75 4.121390481843665e-08 46 0.0007734999999229331
100 4.093102395298634e-08 24 0.0073050000000876025
125 4.5744326702359654e-08 73 0.019387099999903512
150 5.7322266670066874e-08 30 0.006294699999898512
200 3.874743351356943e-08 37 0.007116300000006959
250 5.109062217593784e-08 43 0.009009000000105516
300 4.910873019710894e-08 50 0.012190500000087923
350 5.785905948650014e-08 57 0.015021100000012666


In [48]:
for i, (error, iterations, duration) in enumerate(results_second_1e_7):
    print(n[i], error, iterations, duration)

3 1.7779592572197396e-09 10 0.0002230000000054133
5 5.864354011217591e-09 12 0.00020560000007208146
7 5.077468467664855e-09 14 0.00022439999997914128
10 2.0180271584429476e-09 9 0.00016240000013567624
15 4.0360398071631665e-09 20 0.0003048999999464286
20 2.097904012270478e-09 12 0.00020500000005085894
35 4.023772722526033e-09 31 0.00047629999994569516
50 7.458700993375636e-09 18 0.00034009999990303186
75 6.924245983767158e-09 50 0.0010193999999046355
100 7.395193759102013e-09 26 0.012105900000051406
125 6.6640415774265366e-09 80 0.03094450000003235
150 7.81321330829731e-09 33 0.01661999999987529
200 7.573491680294441e-09 40 0.01986039999997047
250 5.162610764673508e-09 48 0.022234300000036455
300 6.892964201564234e-09 55 0.026397600000109378
350 5.359332806367793e-09 64 0.03171680000014021


Precyzja 1e-15

In [49]:
results_first_1e_15 = JacobiTest.jacobi_n(systems, 1e-15, Condition.FIRST, JacobiTest.get_start_vector_2, 1000)
results_second_1e_15 = JacobiTest.jacobi_n(systems, 1e-15, Condition.SECOND, JacobiTest.get_start_vector_2, 1000)

In [50]:
for i, (error, iterations, duration) in enumerate(results_first_1e_15):
    print(n[i], error, iterations, duration)

3 3.8459253727671276e-16 16 0.00042240000016136037
5 6.080941944488118e-16 20 0.0002987000000302942
7 1.5700924586837752e-16 24 0.00028919999999743595
10 7.021666937153402e-16 14 0.00019720000000233995
15 5.661048867003676e-16 34 0.0003985999999258638
20 6.377745716588144e-16 20 0.00026010000010501244
35 1.336885555457667e-15 52 0.0005986000001030334
50 1.7867348644430382e-15 31 0.0004366000000572967
75 1.9860273225978185e-15 86 0.0017164999999295105
100 1.887379141862766e-15 46 0.01348239999992984
125 2.347275097832166e-15 136 0.03089470000008987
150 3.740314841832019e-15 58 0.012291300000015326
200 5.0719090174402215e-15 73 0.016662700000097175
250 6.654858695507619e-15 96 0.02166409999995267
300 8.022081620984223e-15 111 0.028322200000047815
350 9.236228198169747e-15 999 0.243126600000096


In [51]:
for i, (error, iterations, duration) in enumerate(results_second_1e_15):
    print(n[i], error, iterations, duration)

3 0.0 17 0.0005882000000383414
5 1.1102230246251565e-16 21 0.0006063999999241787
7 1.1102230246251565e-16 25 0.0006877000000713451
10 2.7194799110210365e-16 15 0.000494899999921472
15 5.324442579404919e-16 999 0.019126199999845994
20 6.568167990716596e-16 999 0.014515399999936562
35 1.2412670766236366e-15 999 0.01551659999995536
50 1.7199501139797033e-15 999 0.017002700000148252
75 1.870981097424087e-15 999 0.01748019999990902
100 1.7589240639276204e-15 999 0.44399940000016613
125 2.336749147496729e-15 999 0.3827099999998609
150 3.66709875019605e-15 999 0.37021039999990535
200 5.056087786996822e-15 999 0.35598090000007687
250 6.650226650194185e-15 999 0.36850159999994503
300 8.015933253251316e-15 999 0.3655879000000368
350 9.236228198169747e-15 999 0.36109160000000884


Promień spektralny

In [127]:
def spectral_radius(A):
    B = np.diagflat(np.diag(A)).astype(np.float64)
    I = np.eye(A.shape[0]).astype(np.float64)
    M = I - np.linalg.inv(B) @ A
    return max(abs(np.linalg.eigvals(M)))

In [132]:
for i, x in enumerate(map(spectral_radius, map(lambda x : x[0], [JacobiTest().get_system(n) for n in range(1, 500 + 1)]))):
    print("n = {0}, p = {1}".format(i + 1, x))

n = 1, p = 0.0
n = 2, p = 0.041666666666666664
n = 3, p = 0.0765869787016647
n = 4, p = 0.106724645640869
n = 5, p = 0.13328604069582112
n = 6, p = 0.15706336209739008
n = 7, p = 0.17860643375742666
n = 8, p = 0.1983135558449416
n = 9, p = 0.21648316701401726
n = 10, p = 0.2333450543295334
n = 11, p = 0.24908015871441297
n = 12, p = 0.26383366218814525
n = 13, p = 0.2777239327893888
n = 14, p = 0.2908488156028606
n = 15, p = 0.3032901670949366
n = 16, p = 0.3151171935109013
n = 17, p = 0.3263889549414487
n = 18, p = 0.33715627470070875
n = 19, p = 0.3474632166951742
n = 20, p = 0.35734824359906625
n = 21, p = 0.3668451355778974
n = 22, p = 0.37598372689889514
n = 23, p = 0.3847905023018279
n = 24, p = 0.39328908414344915
n = 25, p = 0.40150063358230315
n = 26, p = 0.4094441834658544
n = 27, p = 0.4171369164733274
n = 28, p = 0.4245943990193639
n = 29, p = 0.43183077913667955
n = 30, p = 0.4388589548223364
n = 31, p = 0.4456907180055828
n = 32, p = 0.4523368782706767
n = 33, p = 0.45880

In [61]:
n_4 = JacobiTest.get_system(4)
print(n_4[0])

[[8.         0.33333333 0.25       0.2       ]
 [0.33333333 8.         0.33333333 0.25      ]
 [0.25       0.33333333 8.         0.33333333]
 [0.2        0.25       0.33333333 8.        ]]


In [62]:
n_3 = JacobiTest.get_system(3)
print(n_3[0])

[[8.         0.33333333 0.25      ]
 [0.33333333 8.         0.33333333]
 [0.25       0.33333333 8.        ]]
